In [2]:
import json
import ollama
from ollama import Options


def extract_question_from_html(html_content):
    response = ollama.generate(
        model='gemma3:27b',
        prompt=f'''
             You are an expert in extracting exam questions from HTML content.
            
             I will provide you with HTML content containing an exam question and its multiple-choice options.
             Your task is to extract the question text and the options as separate items.
            
             HTML content:
             ```
             {html_content}
             ```
            
             Extract and return ONLY in the following JSON format:
             {{
                 "question": "The full question text here without any HTML tags",
                 "options": ["Option 1", "Option 2", "Option 3", "Option 4", "Option 5"]
             }}
            
             Guidelines:
             - Extract the complete question text, including any context or paragraphs preceding the actual question
             - Extract all options exactly as they appear
             - Return valid JSON only, nothing else
             - Remove any HTML tags from the question text and options
             - If you cannot find a question or options, return an empty question or options array
             - Do not include option letters (A, B, C, etc.) in the option text unless they are part of the actual content
             ''',
        options=Options(seed=42)
    )
    result = response['response']
    # Find the pattern that looks like valid JSON using string manipulation
    start_idx = result.find('{')
    end_idx = result.rfind('}') + 1
    if start_idx >= 0 and end_idx > start_idx:
        json_str = result[start_idx:end_idx]
        parsed_json = json.loads(json_str)
        return parsed_json
    raise ValueError("Could not extract valid question and options from response")



In [ ]:
filePaths = [
    'fizik_9_questions.json',
    'fl-biyoloji_6_questions.json',
    'fl-biyoloji_7_questions.json',
    'fl-biyoloji_8_questions.json',
    'fl-biyoloji_9_questions.json',
    'fl-fizik_6_questions.json',
    'fl-fizik_7_questions.json',
    'fl-fizik_8_questions.json',
    'fl-fizik_9_questions.json',
    'fl-kimya_6_questions.json',
    'fl-kimya_7_questions.json',
    'fl-kimya_8_questions.json',
    'fl-kimya_9_questions.json',
    'fl-matematik_6_questions.json',
    'fl-matematik_7_questions.json',
    'fl-matematik_8_questions.json',
    'fl-matematik_9_questions.json',
    'ingilizce_6_questions.json',
    'ingilizce_7_questions.json',
    'ingilizce_8_questions.json',
    'ingilizce_9_questions.json',
    'inkilap-tarihi_9_questions.json',
    'kimya_6_questions.json',
    'kimya_7_questions.json',
    'kimya_8_questions.json', 
    'kimya_9_questions.json',
    'matematik_6_questions.json',
    'matematik_7_questions.json',
    'matematik_8_questions.json',
    'matematik_9_questions.json',
    'tarih_6_questions.json',
    'tarih_7_questions.json',
    'tarih_8_questions.json',
    'tde_6_questions.json',
    'tde_7_questions.json',
    'tde_8_questions.json',
    'tde_9_questions.json',
]

In [4]:

stats = {}

for filename in filePaths:
    with open("questions/" + filename, 'r', encoding='utf-8') as f:
        questions = json.load(f)

        processedQuestions = [q for q in questions if 'answer' in q and q['answer'] < 6 and 'img' not in q['questionAsHtml'].lower()]
        
        # Initialize counters
        image_count = 0
        no_image_count = 0
        total_questions = len(processedQuestions)
        
        for question in processedQuestions:
            print(f"processing question: {processedQuestions.index(question)}/{len(processedQuestions)} id: {question['id']}")
            
            # Skip if question already has options
            if 'options' in question:
                continue
                
            has_image = 'questionAsHtml' in question and 'img' in question['questionAsHtml'].lower()
            hasMoreOptionThan5 = 'answer' in question and question['answer'] > 5
            
            if has_image:
                image_count += 1
            else:
                max_retries = 3
                retry_count = 0
                while retry_count < max_retries:
                    try:
                        # Add timeout of 60 seconds
                        import signal
                        def timeout_handler(signum, frame):
                            raise TimeoutError("Processing took too long")
                            
                        signal.signal(signal.SIGALRM, timeout_handler)
                        signal.alarm(60)  # Set 60 second timeout
                        
                        try:
                            questionResponse = extract_question_from_html(question['questionAsHtml'])
                            signal.alarm(0)  # Disable alarm
                            
                            if questionResponse is not None:
                                print(f"questionResponse: {questionResponse}")
                                question['question'] = questionResponse['question']
                                question['options'] = questionResponse['options']
                                
                                # Save after each successful question processing
                                with open("questions/" + filename, 'w', encoding='utf-8') as f:
                                    json.dump(processedQuestions, f, ensure_ascii=False, indent=2)
                                    
                                break
                            else:
                                print(f"Failed attempt {retry_count + 1} of {max_retries}")
                                retry_count += 1
                                
                        except TimeoutError:
                            print("Question processing timed out after 60 seconds")
                            retry_count += 1
                            signal.alarm(0)  # Disable alarm
                            
                    except Exception as e:
                        print(f"Error on attempt {retry_count + 1}: {str(e)}")
                        retry_count += 1
                        if retry_count == max_retries: 
                            print(f"Failed to process question after {max_retries} attempts")
                 
        # Add stats to dict if there are any href tags
        if image_count > 0:
            stats[filename] = { 
                'total_questions': total_questions,
                'image_count': image_count,
                'no_image_count': no_image_count
            }

processing question: 0/214 id: 23280
processing question: 1/214 id: 23282
processing question: 2/214 id: 435
processing question: 3/214 id: 437
processing question: 4/214 id: 441
processing question: 5/214 id: 445
processing question: 6/214 id: 446
processing question: 7/214 id: 450
processing question: 8/214 id: 451
processing question: 9/214 id: 23278
processing question: 10/214 id: 23279
processing question: 11/214 id: 23285
processing question: 12/214 id: 28347
processing question: 13/214 id: 28348
processing question: 14/214 id: 28350
processing question: 15/214 id: 28355
processing question: 16/214 id: 28357
processing question: 17/214 id: 438
processing question: 18/214 id: 439
processing question: 19/214 id: 440
processing question: 20/214 id: 442
processing question: 21/214 id: 443
processing question: 22/214 id: 444
processing question: 23/214 id: 448
processing question: 24/214 id: 449
processing question: 25/214 id: 23276
Error on attempt 1: Could not extract valid question

KeyboardInterrupt: 